In [24]:
# this is my recreation of the NOAA Weather Data - JFK Jupyter notebook at https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/data-preview/notebooks.html
# this is to help me get familiar with jupyter notebooks

# <a id='toc1_'></a>[Cleaning NOAA Weather Data of JFK Airport](#toc0_)
This notebook relates to the NOAA Weather Dataset - JFK Airport (New York). The dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature and wind speed) collected at JFK airport. This dataset can be obtained for free from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/).

In this notebook, we clean this raw dataset by:

 - removing redundant columns and preserving only key numeric columns
 - converting and cleaning data where required
 - creating a fixed time interval between observations (this aids later with time series analysis)
 - filling missing values
 - encoding certain weather features
 

**Table of contents**<a id='toc0_'></a>    
- [Cleaning NOAA Weather Data of JFK Airport](#toc1_)    
- [Prerequisites](#toc2_)    
  - [Modules](#toc2_1_)    
- [Read The Raw Data](#toc3_)    
- [Clean The Data](#toc4_)    
  - [Select Data Columns](#toc4_1_)    
  - [Clean up precipitation column](#toc4_2_)    
  - [Convert columns to numerical types](#toc4_3_)    
  - [Reformat and process the data](#toc4_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Prerequisites](#toc0_)
Before you run this notebook, complete the following steps:

 - import required modules

## <a id='toc2_1_'></a>[Modules](#toc0_)

In [26]:
import pandas as pd
import numpy as np
import sys
import re
# set max display size for tables with pandas
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# <a id='toc3_'></a>[Read The Raw Data](#toc0_)
Start by reading in the raw dataset, displaying the first few rows of the data frame, and taking a look at the columns and column types present

In [28]:
# define file name and load data
DATA_PATH = '0 - Data/noaa-weather-data-jfk-airport/jfk_weather.csv'
data = pd.read_csv(DATA_PATH, parse_dates=['DATE'])
data.head()    

/var/folders/46/2dh5jv154w30kflp_krsqmz00000gn/T/ipykernel_52913/1273507718.py:3: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,20,22,23,25,40,63,64,67,68,69,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(DATA_PATH, parse_dates=['DATE'])


,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,...,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 00:51:00,FM-15,FEW:02 7 SCT:04 13 BKN:07 29,6.00,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,33.0,0.6,32.0,0.1,31,-0.6,92.0,0.0,000,NaN,29.97,8.0,NaN,29.99,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 01:00:00,FM-12,NaN,5.59,||PL:79,33.0,0.6,32.0,0.1,31,-0.6,92.0,0.0,000,NaN,29.96,8.0,+0.05,29.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 01:51:00,FM-15,FEW:02 6 SCT:04 13 OVC:08 35,6.00,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,33.0,0.6,33.0,0.3,32,0.0,96.0,0.0,000,NaN,29.97,NaN,NaN,29.99,0.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 02:03:00,FM-16,FEW:02 6 SCT:04 13 OVC:08 35,6.00,-RA:02 BR:1 |RA:61 |RA:61,34.0,1.0,33.0,0.7,32,0.0,93.0,0.0,000,NaN,29.97,NaN,NaN,NaN,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 02:28:00,FM-16,BKN:07 7 BKN:07 15 OVC:08 35,5.00,-RA:02 BR:1 |RA:61 |RA:61,34.0,1.0,33.0,0.7,32,0.0,93.0,0.0,000,NaN,29.97,NaN,NaN,NaN,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data.dtypes

STATION                               object
STATION_NAME                          object
ELEVATION                            float64
LATITUDE                             float64
LONGITUDE                            float64
                                      ...   
MonthlyTotalCoolingDegreeDays         object
MonthlyDeptFromNormalHeatingDD        object
MonthlyDeptFromNormalCoolingDD        object
MonthlyTotalSeasonToDateHeatingDD    float64
MonthlyTotalSeasonToDateCoolingDD    float64
Length: 90, dtype: object

# <a id='toc4_'></a>[Clean The Data](#toc0_)
As you can see above, there are a lot of fields which are non-numerical - usually these will be fields that contain text or categorical data, e.g. `HOURLYSKYCONDITIONS`.

There are also fields - such as the main temperature field of interest `HOURLYDRYBULBTEMPF` - that we expect to be numerical, but are instead object type. This often indicates that there may be missing (or null) values, or some other unusual readings that we may have to deal with (since otherwise the field would have been fully parsed as a numerical data type).

In addition, some fields relate to hourly observations, while others relate to daily or monthly intervals. For purposes of later exploratory data analysis, we will restrict the dataset to a certain subset of numerical fields that relate to hourly observations.

In this section, we refer to the [NOAA Local Climatological Data Documentation](https://data.noaa.gov/dataset/dataset/u-s-local-climatological-data-lcd/resource/ee7381ea-647a-434f-8cfa-81202b9b4c05) to describe the fields and meaning of various values.

## <a id='toc4_1_'></a>[Select Data Columns](#toc0_)
First, we select only the subset of data columns of interest and inspect the column types

In [25]:
# choose what columns to import from the raw data
column_subset = [
    'DATE',
    'HOURLYVISIBILITY',
    'HOURLYDRYBULBTEMPF',
    'HOURLYWETBULBTEMPF',
    'HOURLYDewPointTempF',
    'HOURLYRelativeHumidity',
    'HOURLYWindSpeed',
    'HOURLYWindDirection',
    'HOURLYStationPressure',
    'HOURLYPressureTendency',
    'HOURLYSeaLevelPressure',
    'HOURLYPrecip',
    'HOURLYAltimeterSetting'
]

# filter dataset by relevant columns
data_hourly = data[column_subset]
# set date index
data_hourly = data_hourly.set_index(pd.DatetimeIndex(data_hourly['DATE']))
data_hourly.drop(['DATE'], axis=1, inplace=True)
data_hourly.replace(to_replace='*', value=np.nan, inplace=True)
data_hourly.head

<bound method NDFrame.head of                     HOURLYVISIBILITY HOURLYDRYBULBTEMPF HOURLYWETBULBTEMPF  \
DATE                                                                         
2010-01-01 00:51:00             6.00               33.0               32.0   
2010-01-01 01:00:00             5.59               33.0               32.0   
2010-01-01 01:51:00             6.00               33.0               33.0   
2010-01-01 02:03:00             6.00               34.0               33.0   
2010-01-01 02:28:00             5.00               34.0               33.0   
...                              ...                ...                ...   
2018-07-27 20:26:00            10.00                 70                 69   
2018-07-27 20:51:00            10.00                 71                 70   
2018-07-27 21:51:00            10.00                 72                 71   
2018-07-27 22:51:00            10.00                 72                 71   
2018-07-27 23:59:00              N

In [30]:
data_hourly.dtypes

HOURLYVISIBILITY           object
HOURLYDRYBULBTEMPF         object
HOURLYWETBULBTEMPF         object
HOURLYDewPointTempF        object
HOURLYRelativeHumidity     object
HOURLYWindSpeed            object
HOURLYWindDirection        object
HOURLYStationPressure      object
HOURLYPressureTendency    float64
HOURLYSeaLevelPressure     object
HOURLYPrecip               object
HOURLYAltimeterSetting     object
dtype: object

## <a id='toc4_2_'></a>[Clean up precipitation column](#toc0_)
From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NaN` and `T` values. `T` specifies *trace amounts of precipitation*, while `NaN` means *not a number*, and is used to denote missing values.

We can also inspect the unique values present for the field.

In [29]:
data_hourly['HOURLYPrecip'].unique()

array(['0.01', nan, '0.02', 'T', '0.00', '0.10', '0.07', '0.03', '0.05',
       '0.15', '0.06', '0.08', '0.04', '0.09', '0.11', '0.20', '0.13',
       '0.14', '0.05s', '0.12', '0.24', '0.02s', '0.17', '0.16', '0.47',
       '0.28', '0.32', '0.19', '0.22', '0.18', '0.26', '0.23', '0.30',
       '0.33', '0.04s', '0.36', '0.40', '0.41', '0.51', '0.50', '0.68s',
       '0.21', '0.35', '0.54', '0.01s', '0.34', '0.94', '1.14', '1.18',
       '1.15', '0.65', '0.74', '0.39', '0.67', '0.29', '0.46', '0.87',
       '1.10', '0.52', '0.08s', '0.37', '0.42', '0.03s', '0.09s', '0.11s',
       '0.10s', '0.25', '0.33s', '0.13s', '0.38', '0.64s', '0.06s',
       '0.17s', '0.69s', '0.73', '2.41', '0.57', '0.84', '0.86', '0.27',
       '0.60', '0.45', '0.75', '1.26', '0.59', '0.68', '0.79', '0.70',
       '0.44', '0.43', '0.53', '0.37s', '0.77', '0.85', '0.93', '0.55',
       '0.56', '1.52', '1.05', '1.76', '0.07s', '0.63', '1.19', '0.12s',
       '0.18s', '0.58', '0.48', '0.66', '0.29s', '0.71', '0.20s'

We can see that some values end with an `s` (indicating snow), while there is a strange value `0.020.01s` which appears to be an error of some sort. To deal with `T` values, we will set the observation to be `0`. We will also replace the erroneous value `0.020.01s` with `NaN`.

In [32]:
# Fix imported data
data_hourly['HOURLYPrecip'].replace(to_replace='T', value='0.00', inplace=True)
data_hourly['HOURLYPrecip'].replace('0.020.01s', np.nan, inplace=True)

## <a id='toc4_3_'></a>[Convert columns to numerical types](#toc0_)
Next, we will convert string columns that refer to numerical values to numerical types. For columns such as `HOURLYPrecip`, we first also drop the non-numerical parts of the value (the `s` character).

In [33]:
# Set of columns to convert
messy_columns = column_subset[1:]

# Convert columns to float32 type
for i in messy_columns:
    data_hourly[i] = data_hourly[i].apply(lambda x: re.sub('[^0-9,.-]', '', x) if type(x) == str else x).replace('', np.nan).astype('float32')

We can now see that all fields have the numerical data type

In [34]:
print(data_hourly.info())
print()
data_hourly.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 114545 entries, 2010-01-01 00:51:00 to 2018-07-27 23:59:00
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   HOURLYVISIBILITY        96464 non-null   float32
 1   HOURLYDRYBULBTEMPF      111403 non-null  float32
 2   HOURLYWETBULBTEMPF      111245 non-null  float32
 3   HOURLYDewPointTempF     111395 non-null  float32
 4   HOURLYRelativeHumidity  111395 non-null  float32
 5   HOURLYWindSpeed         111358 non-null  float32
 6   HOURLYWindDirection     110444 non-null  float32
 7   HOURLYStationPressure   111256 non-null  float32
 8   HOURLYPressureTendency  48038 non-null   float32
 9   HOURLYSeaLevelPressure  99522 non-null   float32
 10  HOURLYPrecip            79762 non-null   float32
 11  HOURLYAltimeterSetting  86878 non-null   float32
dtypes: float32(12)
memory usage: 6.1 MB
None



,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPressureTendency,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting
DATE,,,,,,,,,,,,
2010-01-01 00:51:00,6.00,33.0,32.0,31.0,92.0,0.0,0.0,29.969999,8.0,29.99,0.01,29.99
2010-01-01 01:00:00,5.59,33.0,32.0,31.0,92.0,0.0,0.0,29.959999,8.0,29.99,NaN,NaN
2010-01-01 01:51:00,6.00,33.0,33.0,32.0,96.0,0.0,0.0,29.969999,NaN,29.99,0.02,29.99
2010-01-01 02:03:00,6.00,34.0,33.0,32.0,93.0,0.0,0.0,29.969999,NaN,NaN,0.00,29.99
2010-01-01 02:28:00,5.00,34.0,33.0,32.0,93.0,0.0,0.0,29.969999,NaN,NaN,0.00,29.99


## <a id='toc4_4_'></a>[Reformat and process the data](#toc0_)
Next, we will clean up some of the data columns to ensure their values fall within the parameters defined by the NOAA documentation (referred to above).

In [35]:
# Generate summary statistics for each column
data_hourly.describe()

,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPressureTendency,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting
count,96464.000000,111403.000000,111245.000000,111395.000000,111395.000000,111358.000000,110444.000000,111256.000000,48038.000000,99522.000000,79762.000000,86878.000000
mean,8.758805,55.310612,49.703098,43.309196,67.132141,11.268045,192.511597,29.990641,4.388963,30.026194,0.007032,30.009533
std,2.742651,17.169744,16.062113,19.357059,20.278572,6.175006,107.606514,0.238452,2.741795,0.233693,0.042636,0.239821
min,0.000000,1.000000,-1.000000,-22.000000,8.000000,0.000000,0.000000,28.490000,0.000000,28.540001,0.000000,28.510000
25%,9.940000,42.000000,37.000000,29.000000,51.000000,7.000000,110.000000,29.850000,2.000000,29.879999,0.000000,29.870001
50%,10.000000,56.000000,50.000000,45.000000,69.000000,10.000000,200.000000,29.990000,5.000000,30.020000,0.000000,30.010000
75%,10.000000,70.000000,64.000000,60.000000,85.000000,15.000000,280.000000,30.139999,7.000000,30.170000,0.000000,30.160000
max,99.419998,102.000000,85.000000,84.000000,100.000000,53.000000,360.000000,30.830000,8.000000,30.850000,2.410000,30.850000


According to the documentation, the `HOURLYPressureTendency` field should be an integer value in the range `[0, 8]`. Let's check if this condition holds for this dataset.

In [36]:
# Check if categorical variable HOURLYPressureTendency ever has a non-integer entry outside the bounds of 0-8.
cond = len(data_hourly[-data_hourly['HOURLYPressureTendency'].isin(list(range(0,9)) + [np.nan])])
print('HOURLYPressureTendency should be between 0 and 8: {}'.format(cond==0))

HOURLYPressureTendency should be between 0 and 8: True


The `HOURLYVISIBILITY` should be an integer between the range `[0, 10]`. Let's check this condition too.

In [37]:
# Hourly Visibility should be between 0 and 10
data_hourly[(data_hourly['HOURLYVISIBILITY'] < 0) | (data_hourly['HOURLYVISIBILITY'] > 10)]

,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPressureTendency,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting
DATE,,,,,,,,,,,,
2011-10-16 11:51:00,14.000000,68.0,53.0,37.0,33.0,16.0,250.0,29.85,NaN,NaN,NaN,29.870001
2015-06-21 17:45:00,99.419998,79.0,72.0,68.0,70.0,37.0,310.0,29.74,NaN,NaN,0.0,29.760000


We find a couple values outside of the range. These must be spurious data observations and we handle them by replacing them with `NaN`.

In [38]:
# Replace any hourly visibility figure outside these bounds with NaN
data_hourly.loc[data_hourly['HOURLYVISIBILITY'] > 10, 'HOURLYVISIBILITY'] = np.nan

# Hourly visibility should be between 0 and 10
cond = len(data_hourly[(data_hourly['HOURLYVISIBILITY'] < 0) | (data_hourly['HOURLYVISIBILITY'] > 10)])
print('Hourly Visibility should be between 0 and 10: {}'.format(cond == 0))

Hourly Visibility should be between 0 and 10: True
